In [1]:
import os
import json
import codecs

class_name_dic = {
  "0": "背景",
  "1": "边异常",
  "2": "角异常",
  "3": "白色点瑕疵",
  "4": "浅色块瑕疵",
  "5": "深色点块瑕疵",
  "6": "光圈瑕疵"
 }
rawImgDir='tile_round1_train_20201231/train_imgs/'
rawLabelDir='tile_round1_train_20201231/train_annos.json'
anno_dir='voc/Annotations/'
if not os.path.exists(anno_dir):
    os.makedirs(anno_dir)
with open(rawLabelDir) as f:
    annos=json.load(f)

#
image_ann={}
for i in range(len(annos)):
    anno=annos[i]
    name = anno['name']
    if name not in image_ann:
        image_ann[name]=[]
    image_ann[name].append(i)
#
for name in image_ann.keys():
    indexs=image_ann[name]
    height, width = annos[indexs[0]]["image_height"], annos[indexs[0]]["image_width"]
    #
    with codecs.open(anno_dir + name[:-4] + '.xml', 'w', 'utf-8') as xml:
        xml.write('<annotation>\n')
        xml.write('\t<filename>' + name + '</filename>\n')
        xml.write('\t<size>\n')
        xml.write('\t\t<width>' + str(width) + '</width>\n')
        xml.write('\t\t<height>' + str(height) + '</height>\n')
        xml.write('\t\t<depth>' + str(3) + '</depth>\n')
        xml.write('\t</size>\n')
        cnt = 0
        for inx in indexs:
            obj = annos[inx]
            assert name == obj['name']
            bbox = obj['bbox']
            category = obj['category']
            xmin, ymin, xmax, ymax = bbox
            class_name = class_name_dic[str(category)]
            #
            xml.write('\t<object>\n')
            xml.write('\t\t<name>' + class_name + '</name>\n')
            xml.write('\t\t<bndbox>\n')
            xml.write('\t\t\t<xmin>' + str(int(xmin)) + '</xmin>\n')
            xml.write('\t\t\t<ymin>' + str(int(ymin)) + '</ymin>\n')
            xml.write('\t\t\t<xmax>' + str(int(xmax)) + '</xmax>\n')
            xml.write('\t\t\t<ymax>' + str(int(ymax)) + '</ymax>\n')
            xml.write('\t\t</bndbox>\n')
            xml.write('\t</object>\n')
            cnt += 1
        assert cnt > 0
        xml.write('</annotation>')

In [2]:
import os
import cv2
import time
import codecs
import xml.etree.ElementTree as ET
from tqdm import tqdm
def get(root, name):
    vars = root.findall(name)
    return vars
def get_and_check(root, name, length):
    vars = root.findall(name)
    if len(vars) == 0:
        raise NotImplementedError('Can not find %s in %s.'%(name, root.tag))
    if length > 0 and len(vars) != length:
        raise NotImplementedError('The size of %s is supposed to be %d, but is %d.'%(name, length, len(vars)))
    if length == 1:
        vars = vars[0]
    return vars

def deal_xml(xml_f):
    tree = ET.parse(xml_f)
    root = tree.getroot()
    object_list=[]
    # 处理每个标注的检测框
    for obj in get(root, 'object'):
        # 取出检测框类别名称
        category = get_and_check(obj, 'name', 1).text
        # 更新类别ID字典
        bndbox = get_and_check(obj, 'bndbox', 1)
        xmin = int(get_and_check(bndbox, 'xmin', 1).text) - 1
        ymin = int(get_and_check(bndbox, 'ymin', 1).text) - 1
        xmax = int(get_and_check(bndbox, 'xmax', 1).text)
        ymax = int(get_and_check(bndbox, 'ymax', 1).text)
        assert (xmax > xmin)
        assert (ymax > ymin)
        o_width = abs(xmax - xmin)
        o_height = abs(ymax - ymin)
        obj_info=[xmin,ymin,xmax,ymax,category]
        object_list.append(obj_info)
    return object_list
def exist_objs(list_1,list_2):
    '''
    list_1:当前slice的图像
    list_2:原图中的所有目标
    return:原图中位于当前slicze中的目标集合
    '''
    return_objs=[]
    s_xmin, s_ymin, s_xmax, s_ymax = list_1[0], list_1[1], list_1[2], list_1[3]
    for vv in list_2:
        xmin, ymin, xmax, ymax,category=vv[0],vv[1],vv[2],vv[3],vv[4]
        if s_xmin<xmin<s_xmax and s_ymin<ymin<s_ymax:#目标点的左上角在切图区域中
            if s_xmin<xmax<s_xmax and s_ymin<ymax<s_ymax:#目标点的右下角在切图区域中
                x_new=xmin-s_xmin
                y_new=ymin-s_ymin
                return_objs.append([x_new,y_new,x_new+(xmax-xmin),y_new+(ymax-ymin),category])

    return return_objs

def make_slice_voc(outpath,exiset_obj_list,sliceHeight=1024, sliceWidth=1024):
    name=outpath.split('/')[-1]
    #
    #
    with codecs.open(os.path.join(slice_voc_dir,  name[:-4] + '.xml'), 'w', 'utf-8') as xml:
        xml.write('<annotation>\n')
        xml.write('\t<filename>' + name + '</filename>\n')
        xml.write('\t<size>\n')
        xml.write('\t\t<width>' + str(sliceWidth) + '</width>\n')
        xml.write('\t\t<height>' + str(sliceHeight) + '</height>\n')
        xml.write('\t\t<depth>' + str(3) + '</depth>\n')
        xml.write('\t</size>\n')
        cnt = 0
        for obj in exiset_obj_list:
            #
            bbox = obj[:4]
            class_name = obj[-1]
            xmin, ymin, xmax, ymax = bbox
            #
            xml.write('\t<object>\n')
            xml.write('\t\t<name>' + class_name + '</name>\n')
            xml.write('\t\t<bndbox>\n')
            xml.write('\t\t\t<xmin>' + str(int(xmin)) + '</xmin>\n')
            xml.write('\t\t\t<ymin>' + str(int(ymin)) + '</ymin>\n')
            xml.write('\t\t\t<xmax>' + str(int(xmax)) + '</xmax>\n')
            xml.write('\t\t\t<ymax>' + str(int(ymax)) + '</ymax>\n')
            xml.write('\t\t</bndbox>\n')
            xml.write('\t</object>\n')
            cnt += 1
        assert cnt > 0
        xml.write('</annotation>')

###############################################################################
def slice_im(image_path, ann_path,out_name, outdir, sliceHeight=1024, sliceWidth=1024,
             zero_frac_thresh=0.2, overlap=0.2, verbose=False):
    #
    object_list=deal_xml(ann_path)

    image0 = cv2.imread(image_path, 1)  # color
    ext = '.' + image_path.split('.')[-1]
    win_h, win_w = image0.shape[:2]
    #print(win_h,win_w)
    # if slice sizes are large than image, pad the edges
    # 避免出现切图的大小比原图还大的情况
    pad = 0
    if sliceHeight > win_h:
        pad = sliceHeight - win_h
    if sliceWidth > win_w:
        pad = max(pad, sliceWidth - win_w)
    # pad the edge of the image with black pixels
    if pad > 0:
        border_color = (0, 0, 0)
        image0 = cv2.copyMakeBorder(image0, pad, pad, pad, pad,
                                    cv2.BORDER_CONSTANT, value=border_color)

    win_size = sliceHeight * sliceWidth

    t0 = time.time()
    n_ims = 0
    n_ims_nonull = 0
    dx = int((1. - overlap) * sliceWidth)
    dy = int((1. - overlap) * sliceHeight)

    for y0 in range(0, image0.shape[0], dy):
        for x0 in range(0, image0.shape[1], dx):
            n_ims += 1
            #
            #这一步确保了不会出现比要切的图像小的图，其实是通过调整最后的overlop来实现的
            #举例:h=6000,w=8192.若使用640来切图,overlop:0.2*640=128,间隔就为512.所以小图的左上角坐标的纵坐标y0依次为:
            #:0,512,1024,....,5120,接下来并非为5632,因为5632+640>6000,所以y0=6000-640
            if y0 + sliceHeight > image0.shape[0]:
                y = image0.shape[0] - sliceHeight
            else:
                y = y0
            if x0 + sliceWidth > image0.shape[1]:
                x = image0.shape[1] - sliceWidth
            else:
                x = x0
            #
            slice_xmax=x+ sliceWidth
            slice_ymax = y + sliceHeight
            exiset_obj_list=exist_objs([x,y,slice_xmax,slice_ymax],object_list)
            if exiset_obj_list!=[]:#如果为空,说明切出来的这一张图不存在目标
                #
                # extract image
                window_c = image0[y:y + sliceHeight, x:x + sliceWidth]
                # get black and white image
                window = cv2.cvtColor(window_c, cv2.COLOR_BGR2GRAY)

                # find threshold that's not black
                #
                ret, thresh1 = cv2.threshold(window, 2, 255, cv2.THRESH_BINARY)
                non_zero_counts = cv2.countNonZero(thresh1)
                zero_counts = win_size - non_zero_counts
                zero_frac = float(zero_counts) / win_size
                # print "zero_frac", zero_fra
                # skip if image is mostly empty
                if zero_frac >= zero_frac_thresh:
                    if verbose:
                        print("Zero frac too high at:", zero_frac)
                    continue
                    # else save
                else:
                    outpath = os.path.join(outdir, out_name + \
                                           '|' + str(y) + '_' + str(x) + '_' + str(sliceHeight) + '_' + str(sliceWidth) + \
                                           '_' + str(pad) + '_' + str(win_w) + '_' + str(win_h) + ext)

                    #
                    if verbose:
                        print("outpath:", outpath)
                    cv2.imwrite(outpath, window_c)
                    n_ims_nonull += 1
                    #------制作新的xml------
                    make_slice_voc(outpath,exiset_obj_list,sliceHeight,sliceWidth)

    #print("Num slices:", n_ims, "Num non-null slices:", n_ims_nonull, \
    #"sliceHeight", sliceHeight, "sliceWidth", sliceWidth)
    #print("Time to slice", image_path, time.time() - t0, "seconds")

if __name__=="__main__":
    use_demo=False
    if use_demo:
        image_path='197_3_t20201119085029342_CAM2.jpg'
        ann_path = '197_3_t20201119085029342_CAM2.xml'
        slice_voc_dir='./slice/annotations_demo'
        outdir = './slice/JPEGImages_demo'
        if not os.path.exists(slice_voc_dir):os.makedirs(slice_voc_dir)
        if not os.path.exists(outdir): os.makedirs(outdir)
        out_name='qyl'
        slice_im(image_path, ann_path,out_name, outdir, sliceHeight=640, sliceWidth=640)
    else:
        raw_images_dir='./tile_round1_train_20201231/train_imgs'#这里的JPEGImages就是原始的图片,建议建立一个原始图片的软连接到这里(或把所有图片复制到voc/JPEGImages下面)
        raw_ann_dir='./voc/Annotations'#上一步convert_
        slice_voc_dir = './slice/annotations'#切出来的标签也保存为voc格式
        outdir = './slice/JPEGImages'
        if not os.path.exists(slice_voc_dir): os.makedirs(slice_voc_dir)
        if not os.path.exists(outdir): os.makedirs(outdir)
        cnt=0
        for per_img_name in tqdm(os.listdir(raw_images_dir)):
            image_path=os.path.join(raw_images_dir,per_img_name)
            ann_path=os.path.join(raw_ann_dir,per_img_name[:-4]+'.xml')
            out_name=str(cnt)
            slice_im(image_path, ann_path, out_name, outdir, sliceHeight=1024, sliceWidth=1024)
            cnt += 1


100%|██████████| 5388/5388 [1:55:29<00:00,  1.29s/it]  


In [ ]:
'''
convert voc to coco format
'''
# -*- coding=utf-8 -*-
#!/usr/bin/python
import sys
import os
import shutil
import numpy as np
import json
import xml.etree.ElementTree as ET
#import mmcv


# 检测框的ID起始值
START_BOUNDING_BOX_ID = 1
# 类别列表无必要预先创建，程序中会根据所有图像中包含的ID来创建并更新
PRE_DEFINE_CATEGORIES ={
  "边异常":1,
  "角异常":2,
  "白色点瑕疵":3,
  "浅色块瑕疵":4,
  "深色点块瑕疵":5,
  "光圈瑕疵":6
 }

def get(root, name):
    vars = root.findall(name)
    return vars


def get_and_check(root, name, length):
    vars = root.findall(name)
    if len(vars) == 0:
        raise NotImplementedError('Can not find %s in %s.'%(name, root.tag))
    if length > 0 and len(vars) != length:
        raise NotImplementedError('The size of %s is supposed to be %d, but is %d.'%(name, length, len(vars)))
    if length == 1:
        vars = vars[0]
    return vars



def convert(xml_list, xml_dir, json_file):
    '''
    :param xml_list: 需要转换的XML文件列表
    :param xml_dir: XML的存储文件夹
    :param json_file: 导出json文件的路径
    :return: None
    '''
    list_fp = xml_list
    image_id=1
    # 标注基本结构
    json_dict = {"images":[],
                 "type": "instances",
                 "annotations": [],
                 "categories": []}
    categories = PRE_DEFINE_CATEGORIES
    bnd_id = START_BOUNDING_BOX_ID
    for line in list_fp:
        line = line.strip()
        print(" Processing {}".format(line))
        # 解析XML
        xml_f = os.path.join(xml_dir, line)
        tree = ET.parse(xml_f)
        root = tree.getroot()
        filename = root.find('filename').text
        # 取出图片名字
        image_id+=1
        size = get_and_check(root, 'size', 1)
        # 图片的基本信息
        width = int(get_and_check(size, 'width', 1).text)
        height = int(get_and_check(size, 'height', 1).text)
        image = {'file_name': filename,
                 'height': height,
                 'width': width,
                 'id':image_id}
        #del image['file_name']
        json_dict['images'].append(image)
        # 处理每个标注的检测框
        for obj in get(root, 'object'):
            # 取出检测框类别名称
            category = get_and_check(obj, 'name', 1).text
            # 更新类别ID字典
            if category not in categories:
                new_id = len(categories)
                categories[category] = new_id
            category_id = categories[category]
            
            print(categories)
            
            bndbox = get_and_check(obj, 'bndbox', 1)
            xmin = int(get_and_check(bndbox, 'xmin', 1).text) - 1
            ymin = int(get_and_check(bndbox, 'ymin', 1).text) - 1
            xmax = int(get_and_check(bndbox, 'xmax', 1).text)
            ymax = int(get_and_check(bndbox, 'ymax', 1).text)
            assert(xmax > xmin)
            assert(ymax > ymin)
            o_width = abs(xmax - xmin)
            o_height = abs(ymax - ymin)
            annotation = dict()
            annotation['area'] = o_width*o_height
            annotation['iscrowd'] = 0
            annotation['image_id'] = image_id
            annotation['bbox'] = [xmin, ymin, o_width, o_height]
            annotation['category_id'] = category_id
            annotation['id'] = bnd_id
            annotation['ignore'] = 0
            # 设置分割数据，点的顺序为逆时针方向
            annotation['segmentation'] = [[xmin,ymin,xmin,ymax,xmax,ymax,xmax,ymin]]

            json_dict['annotations'].append(annotation)
            bnd_id = bnd_id + 1

    # 写入类别ID字典
    for cate, cid in categories.items():
        cat = {'supercategory': 'none', 'id': cid, 'name': cate}
        json_dict['categories'].append(cat)
    # 导出到json
    #mmcv.dump(json_dict, json_file)
    #print(type(json_dict))
    #print(json_dict)
    json_data = json.dumps(json_dict)
    with  open(json_file, 'w') as w:
        w.write(json_data)


if __name__ == '__main__':
    root_path = './'

    if not os.path.exists(os.path.join(root_path,'coco/annotations')):
        os.makedirs(os.path.join(root_path,'coco/annotations'))
    if not os.path.exists(os.path.join(root_path, 'coco/train2017')):
        os.makedirs(os.path.join(root_path, 'coco/train2017'))
    if not os.path.exists(os.path.join(root_path, 'coco/val2017')):
        os.makedirs(os.path.join(root_path, 'coco/val2017'))
    xml_dir = os.path.join(root_path,'slice/annotations') #已知的voc的标注

    xml_labels = os.listdir(xml_dir)
    np.random.shuffle(xml_labels)
    split_point = int(len(xml_labels)/10)

    # validation data
    xml_list = xml_labels[0:split_point]
    json_file = os.path.join(root_path,'coco/annotations/instances_val2017.json')
    convert(xml_list, xml_dir, json_file)
    for xml_file in xml_list:
        img_name = xml_file[:-4] + '.jpg'
        shutil.copy(os.path.join(root_path, 'slice/JPEGImages', img_name),
                    os.path.join(root_path, 'coco/val2017', img_name))
    # train data
    xml_list = xml_labels[split_point:]
    json_file = os.path.join(root_path,'coco/annotations/instances_train2017.json')
    convert(xml_list, xml_dir, json_file)
    for xml_file in xml_list:
        img_name = xml_file[:-4] + '.jpg'
        shutil.copy(os.path.join(root_path, 'slice/JPEGImages', img_name),
                    os.path.join(root_path, 'coco/train2017', img_name))

 Processing 4208|1638_3276_1024_1024_0_8192_6000.xml
{'边异常': 1, '角异常': 2, '白色点瑕疵': 3, '浅色块瑕疵': 4, '深色点块瑕疵': 5, '光圈瑕疵': 6}
 Processing 4884|0_4914_1024_1024_0_8192_6000.xml
{'边异常': 1, '角异常': 2, '白色点瑕疵': 3, '浅色块瑕疵': 4, '深色点块瑕疵': 5, '光圈瑕疵': 6}
{'边异常': 1, '角异常': 2, '白色点瑕疵': 3, '浅色块瑕疵': 4, '深色点块瑕疵': 5, '光圈瑕疵': 6}
 Processing 1801|1638_4914_1024_1024_0_8192_6000.xml
{'边异常': 1, '角异常': 2, '白色点瑕疵': 3, '浅色块瑕疵': 4, '深色点块瑕疵': 5, '光圈瑕疵': 6}
 Processing 3179|2457_1638_1024_1024_0_8192_6000.xml
{'边异常': 1, '角异常': 2, '白色点瑕疵': 3, '浅色块瑕疵': 4, '深色点块瑕疵': 5, '光圈瑕疵': 6}
 Processing 3393|0_1638_1024_1024_0_8192_6000.xml
{'边异常': 1, '角异常': 2, '白色点瑕疵': 3, '浅色块瑕疵': 4, '深色点块瑕疵': 5, '光圈瑕疵': 6}
 Processing 5278|4976_819_1024_1024_0_8192_6000.xml
{'边异常': 1, '角异常': 2, '白色点瑕疵': 3, '浅色块瑕疵': 4, '深色点块瑕疵': 5, '光圈瑕疵': 6}
{'边异常': 1, '角异常': 2, '白色点瑕疵': 3, '浅色块瑕疵': 4, '深色点块瑕疵': 5, '光圈瑕疵': 6}
 Processing 825|0_4914_1024_1024_0_8192_6000.xml
{'边异常': 1, '角异常': 2, '白色点瑕疵': 3, '浅色块瑕疵': 4, '深色点块瑕疵': 5, '光圈瑕疵': 6}
 Processing 3830|16